# OD Pair based analysis WWA

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format

## parameters

In [9]:
BUDGETS = [60,90,120,150,180,210,240]
BUDGET_PRT = BUDGETS[3]
BUDGET_PUT = BUDGET_PRT #trips['Czas_PuT'].quantile(0.5) # a tu juz zakladam ze budzety sa rowne

## read data
* od matrices: Prt, PuT

In [10]:
od_PuT = pd.read_csv('data//MTX_99321.csv')
del od_PuT['Unnamed: 0']
od_PuT.columns = [int(c) for c in od_PuT.columns]
od_PrT = pd.read_csv('data//MTX_99421.csv')
del od_PrT['Unnamed: 0']
od_PrT.columns = [int(c) for c in od_PrT.columns]

In [11]:
od_PrT = od_PrT.stack().reset_index()
od_PrT.columns=['Z_Rejon',"Do_Rejon", 'TripsPrT']
od_PuT = od_PuT.stack().reset_index()
od_PuT.columns=['Z_Rejon',"Do_Rejon", 'TripsPuT']
od = pd.merge(od_PrT, od_PuT, on = ['Z_Rejon', "Do_Rejon"])

* trips

In [12]:
trips = pd.read_csv('data//From_Via_To.csv')
del trips['Unnamed: 0']

## prepare

In [14]:
trips.Czas_PrT = trips.Czas_PrT/60 # przeliczam na minuty
trips.Czas_PuT = trips.Czas_PuT/60 # 
# budzety z zakresu odpowiedniego
trips['time_left_PrT']=BUDGET_PRT - trips.Czas_PrT # ile czasu zostaje ponizej budzetu w POI
trips['time_left_PuT']=BUDGET_PUT - trips.Czas_PuT
trips.Z_Rejon = trips.Z_Rejon.astype(int)
trips.Do_Rejon = trips.Do_Rejon.astype(int)
pd.options.display.float_format = '{:,.1f}'.format

In [15]:
trips = pd.merge(trips,od, on = ['Z_Rejon',"Do_Rejon"])
trips['odPair'] = trips["Z_Rejon"].astype(int)*1000+trips["Do_Rejon"].astype(int)

In [28]:
# tylko polaczenia na ktorych sa podroze
trips = trips[trips.TripsPrT>0.1]

# OD pairs-based analysis:
## number of POI available
- total minutes
- mean/median minutes

In [29]:
print('number of POIs within budget per OD Pair - PuT/PrT')
nPOIs = trips[trips.Czas_PrT<BUDGET_PRT].groupby(by=['Z_Rejon',"Do_Rejon"])['POI'].nunique().to_frame('nPOIsPRT')
nPOIs['nPOIsPUT'] = trips[trips.Czas_PuT<BUDGET_PUT].groupby(by=['Z_Rejon',"Do_Rejon"])['POI'].nunique()
nPOIs.reset_index().head()

number of POIs within budget per OD Pair - PuT/PrT


,Z_Rejon,Do_Rejon,nPOIsPRT,nPOIsPUT
0,2,1,10,10
1,2,3,10,10
2,2,4,10,10
3,2,5,10,10
4,2,6,10,10


## mean/median time available

In [30]:
print("time per OD pair  PrT")
t = trips[trips.Czas_PrT<BUDGET_PRT].groupby(
    by=['Z_Rejon',"Do_Rejon"])['Czas_PrT', 'time_left_PrT'].agg(['sum','mean','median']).reset_index()
t.head()

time per OD pair  PrT


Z_Rejon Do_Rejon Czas_PrT             time_left_PrT             
                        sum mean median           sum  mean median
0       2        1      4.4  0.4    0.5       1,495.6 149.6  149.5
1       2        3      4.4  0.4    0.5       1,495.6 149.6  149.5
2       2        4      4.4  0.4    0.5       1,495.6 149.6  149.5
3       2        5      4.4  0.4    0.5       1,495.6 149.6  149.5
4       2        6      4.4  0.4    0.5       1,495.6 149.6  149.5

In [23]:
print("time per OD pair PuT")
t = trips[trips.Czas_PuT<BUDGET_PUT].groupby(
    by=['Z_Rejon',"Do_Rejon"])['Czas_PuT', 'time_left_PuT'].agg(['sum','mean','median']).reset_index()
t.head()

time per OD pair PuT


Z_Rejon Do_Rejon Czas_PuT             time_left_PuT            
                        sum mean median           sum mean median
0       1        1     91.1 22.8   23.8          31.1  7.8    6.8
1       1        2     86.5 21.6   23.2          35.6  8.9    7.4
2       1        3     89.8 22.4   22.3          32.3  8.1    8.2
3       1        4     93.0 23.3   23.7          29.1  7.3    6.8
4       1        5     66.9 22.3   22.3          24.7  8.2    8.2